In [1]:
import psycopg2
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os
from sqlalchemy import text
from dotenv import load_dotenv

## Mise en dataframe des données

In [2]:
# Mise en DataFrame des csv
# Liste pour stockers les dataframes
df_list = {}
# Dossier avec les csv
data_folder = "data"
# Boucle sur les fichiers du dossier 
for csv in os.listdir(data_folder):
    if csv.endswith(".csv"): # Filtre les fichiers au format csv
        file_path = os.path.join(data_folder, csv)
        df = pd.read_csv(file_path) # Lecture des csv
        # Stockage dans le dictionnaire df_list
        name = os.path.splitext(csv)[0]
        df_list[name] = df
        print(f'{csv} chargé : {df.shape[0]} lignes, {df.shape[1]} colonnes')

olist_order_payments_dataset.csv chargé : 103886 lignes, 5 colonnes
olist_order_reviews_dataset.csv chargé : 99224 lignes, 7 colonnes
olist_products_dataset.csv chargé : 32951 lignes, 9 colonnes
olist_customers_dataset.csv chargé : 99441 lignes, 5 colonnes
olist_sellers_dataset.csv chargé : 3095 lignes, 4 colonnes
olist_orders_dataset.csv chargé : 99441 lignes, 8 colonnes
olist_geolocation_dataset.csv chargé : 1000163 lignes, 5 colonnes
product_category_name_translation.csv chargé : 71 lignes, 2 colonnes
olist_order_items_dataset.csv chargé : 112650 lignes, 7 colonnes


## Connexion à supabase

In [6]:
# Configuration de connexion (à adapter selon votre provider)
load_dotenv()


host = os.getenv('DB_HOST')  #  host Supabase
password = os.getenv('DB_PASSWORD_BIS')
user = os.getenv('DB_USER_BIS')
database = os.getenv('DB_DATABASE')
port = os.getenv('DB_PORT')


# Création de l'engine SQLAlchemy
connection_string = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

# Test de connexion
def test_connection():
    """
    Testez votre connexion à la base

    Étapes :
    1. Utilisez pd.read_sql() pour exécuter "SELECT version()"
    2. Affichez la version PostgreSQL
    3. Gérez les erreurs de connexion
    """
    try:
        df = pd.read_sql('SELECT version();', engine)
        print('Connexion Ok | Version : ', df.iloc[0,0])
    except Exception as e:
        print(f"Erreur de connexion : {e}")
        return False
    return True
test_connection()

Connexion Ok | Version :  PostgreSQL 17.4 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 13.2.0, 64-bit


True

## Création tables


In [8]:
### 🗃️ Création des tables SQL

for table_name, df in df_list.items():
    df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)


## Analyse RFM